# Self-Driving Car Engineer Nanodegree

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, I am solving the second project in the Udacity nanodegree program (Traffic Sign Recognition Classifier)  

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# load data /assuming that data/get_data.sh has been already run
training_file = 'data/train.p'
validation_file = 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

# check if lengths are the same
assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

# print some info
print("Train data shape: {}".format(X_train.shape))
print("Validation data shape: {}".format(X_train.shape))
print("Test data shape: {}".format(X_train.shape))

Train data shape: (34799, 32, 32, 3)
Validation data shape: (34799, 32, 32, 3)
Test data shape: (34799, 32, 32, 3)


4


In [2]:
# show random image
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline

index = randint(0, len(X_train))
image = X_train[index,:,:,:]
sign = y_train[index]

print(y_train[index])
plt.imshow(image)
plt.show()

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [3]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# Number of training examples
n_train = len(X_train)

# Number of validation examples
n_validation = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0,:,:,:].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 34799
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [4]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [5]:
# barplot of the count of signs in the tree sets
from collections import Counter

train_signs_counts = Counter(y_train)
valid_signs_counts = Counter(y_valid)
test_signs_counts = Counter(y_test)

plt.figure(figsize=(15,4))
plt.subplot(131)
plt.bar(list(train_signs_counts.keys()), list(train_signs_counts.values()))
plt.title("Train signs")
plt.subplot(132)
plt.bar(list(valid_signs_counts.keys()), list(valid_signs_counts.values()))
plt.title("Validation signs")
plt.subplot(133)
plt.bar(list(test_signs_counts.keys()), list(test_signs_counts.values()))
plt.title("Test signs")
plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [6]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

# convert rgb images to grayscale
import numpy as np
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

X_train_gray = rgb2gray(X_train)
X_valid_gray = rgb2gray(X_valid)
X_test_gray = rgb2gray(X_test)

# Normalize data
X_train_preprocessed = (X_train - 128)/128
X_valid_preprocessed = (X_valid - 128)/128
X_test_preprocessed = (X_test - 128)/128

#X_train_preprocessed = np.expand_dims(X_train_preprocessed, axis=3)
#X_valid_preprocessed = np.expand_dims(X_valid_preprocessed, axis=3)
#X_test_preprocessed = np.expand_dims(X_test_preprocessed, axis=3)

### Model Architecture

In [7]:
### Define your architecture here.
import tensorflow as tf
from tensorflow.contrib.layers import flatten

tf.reset_default_graph()

def inception2d(x, in_channel, filter_count, mu = 0, sigma = 0.1):
    """
    This method creates an inception layer by combining 1x1, 3x3, 5x5 and avg_pooling layers
    """
    # 1x1 
    one_filter = tf.Variable(tf.truncated_normal([1, 1, in_channel, filter_count], mu, sigma))
    one_by_one = tf.nn.conv2d(x, one_filter, strides=[1, 1, 1, 1], padding='SAME')
    # 3x3
    three_filter = tf.Variable(tf.truncated_normal([3, 3, in_channel, filter_count], mu, sigma))
    three_by_three = tf.nn.conv2d(x, three_filter, strides=[1, 1, 1, 1], padding='SAME')
    # 5x5
    five_filter = tf.Variable(tf.truncated_normal([5, 5, in_channel, filter_count], mu, sigma))
    five_by_five = tf.nn.conv2d(x, five_filter, strides=[1, 1, 1, 1], padding='SAME')
    # avg pooling
    pooling = tf.nn.avg_pool(x, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')
    # concatenate layers
    concat = tf.concat(values=[one_by_one, three_by_three, five_by_five, pooling], concat_dim=3)
    # add bias and relu activation function
    bias = tf.Variable(tf.truncated_normal([3*filter_count + in_channel], mu, sigma))
    output = tf.nn.relu(tf.nn.bias_add(concat, bias)) 
    return output

def Inception(x):
    mu = 0
    sigma = 0.1
    weights = {
        'wc1' : tf.Variable(tf.truncated_normal([5, 5, 3, 6], mu, sigma)),
        'wc2' : tf.Variable(tf.truncated_normal([5, 5, 6, 16], mu, sigma)),
        'fc1' : tf.Variable(tf.truncated_normal([850, 400], mu, sigma)),
        'fc2' : tf.Variable(tf.truncated_normal([400, 80], mu, sigma)),
        'out' : tf.Variable(tf.truncated_normal([80, 43], mu, sigma))
    }
    biases = {
        'bc1' : tf.Variable(tf.truncated_normal([6], mu, sigma)),
        'bc2' : tf.Variable(tf.truncated_normal([16], mu, sigma)),
        'bfc1' : tf.Variable(tf.truncated_normal([400], mu, sigma)),
        'bfc2' : tf.Variable(tf.truncated_normal([80], mu, sigma)),
        'out' : tf.Variable(tf.truncated_normal([43], mu, sigma))
    }
    # Layer 1: convolutional layer
    conv1 = tf.nn.conv2d(x, weights['wc1'], strides=[1, 1, 1, 1], padding='VALID')
    conv1 = tf.nn.bias_add(conv1, biases['bc1'])
    conv1 = tf.nn.relu(conv1)
    # Layer 2: avg pooling
    pool1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Layer 3: convolutional layer
    conv2 = tf.nn.conv2d(pool1, weights['wc2'], strides=[1, 1, 1, 1], padding='VALID')
    conv2 = tf.nn.bias_add(conv2, biases['bc2'])
    conv2 = tf.nn.relu(conv2)
    # Layer 4: avg pooling
    pool2 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Layer 5: Inception 
    incept1 = inception2d(pool2, in_channel=16, filter_count=6)
    # Layer 6: Inception 
    #incept2 = inception2d(incept1, in_channel=34, filter_count=6)
    # Layer 7: flatten
    flat = flatten(incept1)
    # Layer 8: Fully connected
    fc1 = tf.add(tf.matmul(flat, weights['fc1']), biases['bfc1'])
    fc1 = tf.nn.relu(fc1)
    # Layer 8: Fully connected
    fc2 = tf.add(tf.matmul(fc1, weights['fc2']), biases['bfc2'])
    fc2 = tf.nn.relu(fc2)
    # Layer 10: Fully connected
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    
    return out

In [8]:
# define placeholders 
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int32, [None])
y_one_hot = tf.one_hot(y, 43)

In [9]:
# training pipeline
EPOCHS = 10
BATCH_SIZE = 128
learning_rate = 0.001

logits = Inception(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, y_one_hot)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

# model evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [10]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset + BATCH_SIZE], y_data[offset:offset + BATCH_SIZE]
        batch_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += batch_accuracy*len(batch_x)
    return total_accuracy/num_examples

In [11]:
train_writer = tf.summary.FileWriter('./summaries/inception/plot_train')
valid_writer = tf.summary.FileWriter('./summaries/inception/plot_valid')

tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss)
write_op = tf.summary.merge_all()

In [12]:
# train the model
from sklearn.utils import shuffle
import random

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train_preprocessed, y_train = shuffle(X_train_preprocessed, y_train)
        
        for counter, offset in enumerate(range(0, num_examples, BATCH_SIZE)):
            batch_x, batch_y = X_train_preprocessed[offset: offset + BATCH_SIZE], y_train[offset: offset + BATCH_SIZE]
            merge = tf.summary.merge_all()
            batch_loss, batch_acc, _ = sess.run([loss, accuracy, train_step], 
                                                feed_dict={x: batch_x, y: batch_y})
            if counter % 100 == 0:
                print("EPOCH : {:>3}, STEP : {:>5}, Train Accuracy: {:.3f} Train Loss: {:.3f}".
                      format(i+1, counter,batch_acc, batch_loss))
        
        # log training and validation accuracy
        train_index = random.sample(range(num_examples), 5000)
        summary = sess.run(write_op, feed_dict={x: X_train_preprocessed[train_index], y: y_train[train_index]})
        train_writer.add_summary(summary, i)
        train_writer.flush()
        
        summary = sess.run(write_op, feed_dict={x: X_valid_preprocessed, y: y_valid})
        valid_writer.add_summary(summary, i)
        valid_writer.flush()
                
        validation_accuracy = evaluate(X_valid_preprocessed, y_valid)
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))

Training...

EPOCH :   1, STEP :     0, Train Accuracy: 0.008 Train Loss: 3.904
EPOCH :   1, STEP :   100, Train Accuracy: 0.172 Train Loss: 3.073
EPOCH :   1, STEP :   200, Train Accuracy: 0.445 Train Loss: 2.168
Validation Accuracy = 0.360
EPOCH :   2, STEP :     0, Train Accuracy: 0.492 Train Loss: 1.634
EPOCH :   2, STEP :   100, Train Accuracy: 0.727 Train Loss: 1.068
EPOCH :   2, STEP :   200, Train Accuracy: 0.703 Train Loss: 0.940
Validation Accuracy = 0.566
EPOCH :   3, STEP :     0, Train Accuracy: 0.781 Train Loss: 0.729
EPOCH :   3, STEP :   100, Train Accuracy: 0.859 Train Loss: 0.504
EPOCH :   3, STEP :   200, Train Accuracy: 0.836 Train Loss: 0.531
Validation Accuracy = 0.640
EPOCH :   4, STEP :     0, Train Accuracy: 0.867 Train Loss: 0.442
EPOCH :   4, STEP :   100, Train Accuracy: 0.859 Train Loss: 0.486
EPOCH :   4, STEP :   200, Train Accuracy: 0.844 Train Loss: 0.426
Validation Accuracy = 0.665
EPOCH :   5, STEP :     0, Train Accuracy: 0.875 Train Loss: 0.390
EPOC

In [169]:
### Define your architecture here.
import tensorflow as tf
from tensorflow.contrib.layers import flatten

tf.reset_default_graph()

def LeNet(x):
    mu = 0
    sigma = 0.1
    weights = {
        'wc1' : tf.Variable(tf.truncated_normal([5, 5, 3, 6], mu, sigma)),
        'wc2' : tf.Variable(tf.truncated_normal([5, 5, 6, 16], mu, sigma)),
        'fc1' : tf.Variable(tf.truncated_normal([400, 120], mu, sigma)),
        'fc2' : tf.Variable(tf.truncated_normal([120, 80], mu, sigma)),
        'out' : tf.Variable(tf.truncated_normal([80, 43], mu, sigma))
    }
    biases = {
        'bc1' : tf.Variable(tf.truncated_normal([6], mu, sigma)),
        'bc2' : tf.Variable(tf.truncated_normal([16], mu, sigma)),
        'bfc1' : tf.Variable(tf.truncated_normal([120], mu, sigma)),
        'bfc2' : tf.Variable(tf.truncated_normal([80], mu, sigma)),
        'out' : tf.Variable(tf.truncated_normal([43], mu, sigma))
    }
    # Layer 1: convolutional layer
    conv1 = tf.nn.conv2d(x, weights['wc1'], strides=[1, 1, 1, 1], padding='VALID')
    conv1 = tf.nn.bias_add(conv1, biases['bc1'])
    conv1 = tf.nn.relu(conv1)
    
    # Layer 2: avg pooling
    pool1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Layer 3: convolutional layer
    conv2 = tf.nn.conv2d(pool1, weights['wc2'], strides=[1, 1, 1, 1], padding='VALID')
    conv2 = tf.nn.bias_add(conv2, biases['bc2'])
    conv2 = tf.nn.relu(conv2)
    
    # Layer 4: avg pooling
    pool2 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Layer 5: flatten
    flat = flatten(pool2)
    
    # Layer 6: Fully connected
    fc1 = tf.add(tf.matmul(flat, weights['fc1']), biases['bfc1'])
    fc1 = tf.nn.relu(fc1)
    
    # Layer 7: Fully connected
    fc2 = tf.add(tf.matmul(fc1, weights['fc2']), biases['bfc2'])
    fc2 = tf.nn.relu(fc2)
    
    # Layer 8: Fully connected
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
        
    return out

In [170]:
# define placeholders 
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.int32, [None])
y_one_hot = tf.one_hot(y, 43)

In [171]:
# training pipeline
EPOCHS = 10
BATCH_SIZE = 128
learning_rate = 0.001

logits = classifier(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, y_one_hot)
train_loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(train_loss)


In [172]:
# model evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset + BATCH_SIZE], y_data[offset:offset + BATCH_SIZE]
        batch_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += batch_accuracy*len(batch_x)
    return total_accuracy/num_examples

In [173]:
# train the model
from sklearn.utils import shuffle

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        
        for counter, offset in enumerate(range(0, num_examples, BATCH_SIZE)):
            batch_x, batch_y = X_train[offset: offset + BATCH_SIZE], y_train[offset: offset + BATCH_SIZE]
            batch_loss, batch_acc, _ = sess.run([train_loss, accuracy, train_step], feed_dict={x: batch_x, y: batch_y})
            if counter % 100 == 0:
                print("EPOCH : {:>3}, STEP : {:>5}, Train Accuracy: {:.3f} Train Loss: {:.3f}".
                      format(i+1, counter,batch_acc, batch_loss))
                
        validation_accuracy = evaluate(X_valid, y_valid)
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))

Training...

EPOCH :   1, STEP :     0, Train Accuracy: 0.023 Train Loss: 23.931
EPOCH :   1, STEP :   100, Train Accuracy: 0.484 Train Loss: 1.762
EPOCH :   1, STEP :   200, Train Accuracy: 0.773 Train Loss: 0.830
Validation Accuracy = 0.768
EPOCH :   2, STEP :     0, Train Accuracy: 0.852 Train Loss: 0.511
EPOCH :   2, STEP :   100, Train Accuracy: 0.898 Train Loss: 0.348
EPOCH :   2, STEP :   200, Train Accuracy: 0.898 Train Loss: 0.457
Validation Accuracy = 0.856
EPOCH :   3, STEP :     0, Train Accuracy: 0.938 Train Loss: 0.204
EPOCH :   3, STEP :   100, Train Accuracy: 0.969 Train Loss: 0.105
EPOCH :   3, STEP :   200, Train Accuracy: 0.961 Train Loss: 0.135
Validation Accuracy = 0.855
EPOCH :   4, STEP :     0, Train Accuracy: 0.953 Train Loss: 0.149
EPOCH :   4, STEP :   100, Train Accuracy: 0.945 Train Loss: 0.164
EPOCH :   4, STEP :   200, Train Accuracy: 0.961 Train Loss: 0.121
Validation Accuracy = 0.871
EPOCH :   5, STEP :     0, Train Accuracy: 0.977 Train Loss: 0.108
EPO

In [151]:
y_valid.shape

(4410,)

In [140]:
[i for i, x in enumerate(range(0,10,2))]

[0, 1, 2, 3, 4]

In [ ]:
en

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [1]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [3]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [4]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [3]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")